In [6]:
import sys
sys.path.append('/home/zhw272/programming/preprocessing-pipeline/in_development/Will')
sys.path.append('/home/zhw272/programming/preprocessing-pipeline/src')
from cell_extractor.CellDetectorTrainer import CellDetectorTrainer
from cell_extractor.DetectorMetrics import AnnotationProximityTool
import pandas as pd

1.6.0



In [7]:
trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=2300)
trainer.list_detectors()

/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

['detector_round_2_threshold_2300.pkl',
 'detector_round_2_threshold_2200.pkl',
 'detector_round_2_threshold_2100.pkl',
 'detector_round_2.pkl',
 'detector_round_2_threshold_2000.pkl',
 'detector_round_1.pkl',
 'detector_round_2_threshold_2700.pkl']

In [8]:
all_detections = []
for threshold in [2000,2100,2200,2300,2700]:
    trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=threshold)
    detections = trainer.load_detections()
    detections = detections[['row','col','section','predictions']]
    detections['name'] = str(threshold)
    all_detections.append(detections)
all_detections = pd.concat(all_detections)

/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


In [9]:
sures = all_detections[all_detections.predictions==2]
unsures = all_detections[all_detections.predictions==0]

In [10]:
sures = sures.drop('predictions',axis=1)
sures.columns = ['x','y','section','name']

In [11]:
util = AnnotationProximityTool()
util.set_annotations_to_compare(sures)
util.find_equivalent_points()

calculating distance matrix


/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preprocessing-pipeline/in_development/Will/cell_extractor/CellDetectorBase.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tile_information = tile_information.append(tile_informationi,ignore_index=True)
/home/zhw272/programming/preproces

MemoryError: Unable to allocate 29.2 GiB for an array with shape (62614, 62614) and data type float64